## MICCAI Test Pipeline

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from imports import *

/ext3/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-01-28 20:10:03.424446: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 20:10:03.687062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 20:10:04.490766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# overall imports
import importlib
import data

# importlib.reload(data.data_utils)
# importlib.reload(data.data_load)
#importlib.reload(sim.sim_utils)
# importlib.reload(models)

import models
import sim.sim 
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run, open_pickled_results


#### Check job specs

In [ ]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

In [ ]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

In [ ]:
GPUtil.getAvailable()
# if a number is seen a GPU is available

In [ ]:
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

In [ ]:
GPUtil.showUtilization()


In [ ]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

## Start test pipeline <a id="sims"></a>

In [ ]:
def run_simulation_set(model_types=['dynamic_mlp'],
                      cv_types=['spatial', 'random'],
                      parcellations=['S100', 'S400'],
                      connectome_targets=['FC', 'SC'],
                      feature_types=['transcriptome', 'euclidean'],
                      hemisphere='left',
                      random_seeds=[42, 123, 456]):
    """
    Run a set of simulations with different combinations of parameters.
    
    Args:
        model_types (list): List of model types to test
        cv_types (list): List of cross-validation types
        parcellations (list): List of parcellation schemes
        connectome_targets (list): List of connectome targets
        feature_types (list): List of feature types
        hemisphere (str): Brain hemisphere to use ('left', 'right', 'both')
        random_seeds (list): List of random seeds for multiple runs
    """
    
    for seed in random_seeds:
        for model in model_types:
            for cv in cv_types:
                for parc in parcellations:
                    # Set hemisphere based on parcellation
                    current_hemisphere = 'both' if parc == 'S100' else 'left'
                    
                    for target in connectome_targets:
                        for feat in feature_types:
                            # Configure feature type dict
                            feat_dict = [{'transcriptome': None}] if feat == 'transcriptome' else [{'euclidean': None}]
                            
                            print(f"Running simulation with: {model}, {cv}, {parc}, {target}, {feat}, seed={seed}")
                            
                            # Run single simulation
                            single_sim_run(
                                cv_type=cv,
                                random_seed=seed,
                                model_type=model,
                                feature_type=feat_dict,
                                connectome_target=target,
                                use_gpu=True,
                                use_shared_regions=False,
                                test_shared_regions=False,
                                omit_subcortical=False,
                                parcellation=parc,
                                gene_list='0.2',
                                hemisphere=current_hemisphere,
                                search_method=('wandb', 'mse', 10),
                                save_sim=False,
                                track_wandb=True,
                                skip_cv=False
                            )
                            
                            # Clear GPU memory
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()
                            # Clear CPU memory
                            gc.collect()

# Example usage:
run_simulation_set(
    model_types=['dynamic_mlp'],
    cv_types=['random', 'spatial'],
    parcellations=['S100', 'S400'],
    connectome_targets=['FC', 'SC'],
    feature_types=['transcriptome', 'euclidean'],
    hemisphere='both',
    random_seeds=[1, 2]
)